KAYNAKÇA <hr>
https://www.researchgate.net/publication/383198733_Computer_Vision-Based_Human_Body_Posture_Correction_System <hr>
https://github.com/SuriyaaVijay/Posture-Recognition-using-Opencv-and-Mediapipe <hr>
https://pmc.ncbi.nlm.nih.gov/articles/PMC8022631/ <hr>

In [2]:
def turkish_character_fix(text):
    replacements = {
        'ı': 'i',
        'ş': 's',
        'ç': 'c',
        'ğ': 'g',
        'ö': 'o',
        'ü': 'u',
        'İ': 'I',
        'Ş': 'S',
        'Ç': 'C',
        'Ğ': 'G',
        'Ö': 'O',
        'Ü': 'U'
    }
    for original, replacement in replacements.items():
        text = text.replace(original, replacement)
    return text


Versiyon 1 - en stabili

In [7]:
import cv2
import mediapipe as mp
import numpy as np
import time

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
cap = cv2.VideoCapture(0)

is_calibrated = False
calibration_frames = 0
calibration_shoulder_angles = []
calibration_neck_angles = []

last_alert_time = 0
alert_cooldown = 5  

def calculate_angle(point1, point2, point3):
    a = np.array(point1)
    b = np.array(point2)
    c = np.array(point3)

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360.0 - angle

    return angle

def draw_angle(frame, point1, point2, point3, angle, color):
    cv2.putText(frame, str(int(angle)), point2, cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2, cv2.LINE_AA)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        continue

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        left_shoulder = (int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x * frame.shape[1]),
                         int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y * frame.shape[0]))
        right_shoulder = (int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x * frame.shape[1]),
                          int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y * frame.shape[0]))
        left_ear = (int(landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].x * frame.shape[1]),
                    int(landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].y * frame.shape[0]))
        right_ear = (int(landmarks[mp_pose.PoseLandmark.RIGHT_EAR.value].x * frame.shape[1]),
                     int(landmarks[mp_pose.PoseLandmark.RIGHT_EAR.value].y * frame.shape[0]))

        shoulder_angle = calculate_angle(left_shoulder, right_shoulder, (right_shoulder[0], 0))
        neck_angle = calculate_angle(left_ear, left_shoulder, (left_shoulder[0], 0))

        if not is_calibrated and calibration_frames < 30:
            calibration_shoulder_angles.append(shoulder_angle)
            calibration_neck_angles.append(neck_angle)
            calibration_frames += 1
            cv2.putText(frame, f"Kalibre Ediliyor... {calibration_frames}/30", (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
        elif not is_calibrated:
            shoulder_threshold = np.mean(calibration_shoulder_angles) - 3
            neck_threshold = np.mean(calibration_neck_angles) - 3
            is_calibrated = True
            print(f"Kalibrasyon tamamlandi. Omuz limiti: {shoulder_threshold:.1f}, Boyun limiti: {neck_threshold:.1f}")

        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        midpoint = ((left_shoulder[0] + right_shoulder[0]) // 2, (left_shoulder[1] + right_shoulder[1]) // 2)
        draw_angle(frame, left_shoulder, midpoint, (midpoint[0], 0), shoulder_angle, (255, 0, 0))
        draw_angle(frame, left_ear, left_shoulder, (left_shoulder[0], 0), neck_angle, (0, 255, 0))

        if is_calibrated:
            current_time = time.time()
            if shoulder_angle < shoulder_threshold or neck_angle < neck_threshold:
                status = "Kotu Postur"
                color = (0, 0, 255)  
            else:
                status = "Iyi Postur"
                color = (0, 255, 0)  

            cv2.putText(frame, status, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
            cv2.putText(frame, f"Omuz Aci: {shoulder_angle:.1f}/{shoulder_threshold:.1f}", (10, 60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)
            cv2.putText(frame, f"Boyun Aci: {neck_angle:.1f}/{neck_threshold:.1f}", (10, 90), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)

    cv2.imshow('Postur Kontrolu', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Kalibrasyon tamamlandi. Omuz limiti: 82.2, Boyun limiti: 23.2


-------------------------------------------

Versiyon 2 -  hatası sadece sol kulak alması

In [5]:
import cv2
import mediapipe as mp
import numpy as np
import time

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
cap = cv2.VideoCapture(0)

is_calibrated = False
calibration_frames = 0
calibration_neck_distances = []
threshold_distance = 0

last_alert_time = 0
alert_cooldown = 5  

def calculate_forward_head_distance(left_ear, left_shoulder):
    return np.abs(left_ear[0] - left_shoulder[0])

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        continue

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        left_shoulder = (int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x * frame.shape[1]),
                         int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y * frame.shape[0]))
        left_ear = (int(landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].x * frame.shape[1]),
                    int(landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].y * frame.shape[0]))

        head_forward_distance = calculate_forward_head_distance(left_ear, left_shoulder)

        if not is_calibrated and calibration_frames < 30:
            calibration_neck_distances.append(head_forward_distance)
            calibration_frames += 1
            cv2.putText(frame, f"Kalibre Ediliyor... {calibration_frames}/30", (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
        elif not is_calibrated:
            threshold_distance = np.mean(calibration_neck_distances) + 15 
            is_calibrated = True
            print(f"Kalibrasyon tamamlandi. Baş-omuz eşik mesafesi: {threshold_distance:.1f}")

        if is_calibrated:
            current_time = time.time()
            if head_forward_distance > threshold_distance:
                status = "Kafa Cok Onda"  
                color = (0, 0, 255)  
            else:
                status = "Iyi Postur"
                color = (0, 255, 0) 
            cv2.putText(frame, status, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
            cv2.putText(frame, f"Kafa-Omuz Mesafe: {head_forward_distance:.1f}/{threshold_distance:.1f}", (10, 60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)

    cv2.imshow('Postur Kontrolu', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Kalibrasyon tamamlandi. Baş-omuz eşik mesafesi: 91.6


Versiyon 3 - stabil fakat halen öne eğilme posture bozukluğunda hatalı

In [10]:
import cv2
import mediapipe as mp
import numpy as np
import time

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
cap = cv2.VideoCapture(0)

is_calibrated = False
calibration_frames = 0
calibration_shoulder_angles = []
calibration_neck_angles = []
calibration_head_distances = []

last_alert_time = 0
alert_cooldown = 5  

def calculate_angle(point1, point2, point3):
    a = np.array(point1)
    b = np.array(point2)
    c = np.array(point3)

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360.0 - angle

    return angle

def calculate_head_distance(ear, shoulder):
    return np.abs(ear[0] - shoulder[0])

def draw_angle(frame, point1, point2, point3, angle, color):
    cv2.putText(frame, str(int(angle)), point2, cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2, cv2.LINE_AA)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        continue

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        left_shoulder = (int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x * frame.shape[1]),
                         int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y * frame.shape[0]))
        right_shoulder = (int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x * frame.shape[1]),
                          int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y * frame.shape[0]))
        left_ear = (int(landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].x * frame.shape[1]),
                    int(landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].y * frame.shape[0]))
        right_ear = (int(landmarks[mp_pose.PoseLandmark.RIGHT_EAR.value].x * frame.shape[1]),
                     int(landmarks[mp_pose.PoseLandmark.RIGHT_EAR.value].y * frame.shape[0]))

        shoulder_angle = calculate_angle(left_shoulder, right_shoulder, (right_shoulder[0], 0))
        neck_angle = calculate_angle(left_ear, left_shoulder, (left_shoulder[0], 0))

        head_distance = calculate_head_distance(left_ear, left_shoulder)

        if not is_calibrated and calibration_frames < 30:
            calibration_shoulder_angles.append(shoulder_angle)
            calibration_neck_angles.append(neck_angle)
            calibration_head_distances.append(head_distance)
            calibration_frames += 1
            cv2.putText(frame, f"Kalibre Ediliyor... {calibration_frames}/30", (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
        elif not is_calibrated:
            shoulder_threshold = np.mean(calibration_shoulder_angles) - 3
            neck_threshold = np.mean(calibration_neck_angles) - 3
            head_distance_threshold = np.mean(calibration_head_distances) + 10 
            is_calibrated = True
            print(f"Kalibrasyon tamamlandi. Omuz: {shoulder_threshold:.1f}, Boyun: {neck_threshold:.1f}, Kafa Mesafesi: {head_distance_threshold:.1f}")

        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        midpoint = ((left_shoulder[0] + right_shoulder[0]) // 2, (left_shoulder[1] + right_shoulder[1]) // 2)
        draw_angle(frame, left_shoulder, midpoint, (midpoint[0], 0), shoulder_angle, (255, 0, 0))
        draw_angle(frame, left_ear, left_shoulder, (left_shoulder[0], 0), neck_angle, (0, 255, 0))

        if is_calibrated:
            current_time = time.time()
            if shoulder_angle < shoulder_threshold or neck_angle < neck_threshold or head_distance > head_distance_threshold:
                status = "Kotu Postur" 
                color = (0, 0, 255)  
            else:
                status = "Iyi Postur"
                color = (0, 255, 0)  

            cv2.putText(frame, status, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
            cv2.putText(frame, f"Omuz Aci: {shoulder_angle:.1f}/{shoulder_threshold:.1f}", (10, 60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)
            cv2.putText(frame, f"Boyun Aci: {neck_angle:.1f}/{neck_threshold:.1f}", (10, 90), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)
            cv2.putText(frame, f"Kafa Mesafesi: {head_distance:.1f}/{head_distance_threshold:.1f}", (10, 120), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)

    cv2.imshow('Postur Kontrolu', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Kalibrasyon tamamlandi. Omuz: 82.3, Boyun: 23.6, Kafa Mesafesi: 88.9


Versiyon 4  - Şekil Değiştirildi

In [2]:
import cv2
import mediapipe as mp
import numpy as np
import time

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
cap = cv2.VideoCapture(0)

is_calibrated = False
calibration_frames = 0
calibration_shoulder_angles = []
calibration_neck_angles = []
calibration_head_distances = []

last_alert_time = 0
alert_cooldown = 5  

def calculate_angle(point1, point2, point3):
    a = np.array(point1)
    b = np.array(point2)
    c = np.array(point3)

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360.0 - angle

    return angle

def calculate_head_distance(ear, shoulder):
    return np.abs(ear[0] - shoulder[0])

def draw_angle(frame, point1, point2, point3, angle, color):
    cv2.putText(frame, str(int(angle)), point2, cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2, cv2.LINE_AA)

def draw_shapes(frame, left_shoulder, right_shoulder, left_ear, right_ear):
    midpoint = ((left_shoulder[0] + right_shoulder[0]) // 2, (left_shoulder[1] + right_shoulder[1]) // 2)

    cv2.line(frame, left_shoulder, midpoint, (255, 255, 255), 2)
    cv2.line(frame, right_shoulder, midpoint, (255, 255, 255), 2)
    
    cv2.line(frame, left_shoulder, left_ear, (255, 255, 255), 2)
    cv2.line(frame, right_shoulder, right_ear, (255, 255, 255), 2)
    
    upper_midpoint = (midpoint[0], midpoint[1] - 100)  
    cv2.line(frame, midpoint, upper_midpoint, (255, 255, 255), 2)
    cv2.line(frame, left_ear, upper_midpoint, (255, 255, 255), 2)
    cv2.line(frame, right_ear, upper_midpoint, (255, 255, 255), 2)

    cv2.circle(frame, left_shoulder, 5, (255, 255, 255), -1)
    cv2.circle(frame, right_shoulder, 5, (255, 255, 255), -1)
    cv2.circle(frame, midpoint, 5, (255, 255, 255), -1)
    cv2.circle(frame, upper_midpoint, 5, (255, 255, 255), -1)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        continue

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        left_shoulder = (int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x * frame.shape[1]),
                         int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y * frame.shape[0]))
        right_shoulder = (int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x * frame.shape[1]),
                          int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y * frame.shape[0]))
        left_ear = (int(landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].x * frame.shape[1]),
                    int(landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].y * frame.shape[0]))
        right_ear = (int(landmarks[mp_pose.PoseLandmark.RIGHT_EAR.value].x * frame.shape[1]),
                     int(landmarks[mp_pose.PoseLandmark.RIGHT_EAR.value].y * frame.shape[0]))

        shoulder_angle = calculate_angle(left_shoulder, right_shoulder, (right_shoulder[0], 0))
        neck_angle = calculate_angle(left_ear, left_shoulder, (left_shoulder[0], 0))
        head_distance = calculate_head_distance(left_ear, left_shoulder)

        if not is_calibrated and calibration_frames < 30:
            calibration_shoulder_angles.append(shoulder_angle)
            calibration_neck_angles.append(neck_angle)
            calibration_head_distances.append(head_distance)
            calibration_frames += 1
            cv2.putText(frame, f"Kalibre Ediliyor... {calibration_frames}/30", (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
        elif not is_calibrated:
            shoulder_threshold = np.mean(calibration_shoulder_angles) - 3
            neck_threshold = np.mean(calibration_neck_angles) - 3
            head_distance_threshold = np.mean(calibration_head_distances) + 10 
            is_calibrated = True
            print(f"Kalibrasyon tamamlandi. Omuz: {shoulder_threshold:.1f}, Boyun: {neck_threshold:.1f}, Kafa Mesafesi: {head_distance_threshold:.1f}")

        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        draw_shapes(frame, left_shoulder, right_shoulder, left_ear, right_ear)

        midpoint = ((left_shoulder[0] + right_shoulder[0]) // 2, (left_shoulder[1] + right_shoulder[1]) // 2)
        draw_angle(frame, left_shoulder, midpoint, (midpoint[0], 0), shoulder_angle, (255, 0, 0))
        draw_angle(frame, left_ear, left_shoulder, (left_shoulder[0], 0), neck_angle, (0, 255, 0))

        if is_calibrated:
            current_time = time.time()
            if shoulder_angle < shoulder_threshold or neck_angle < neck_threshold or head_distance > head_distance_threshold:
                status = "Kötü Postür" 
                color = (0, 0, 255)  
            else:
                status = "İyi Postür"
                color = (0, 255, 0)  

            cv2.putText(frame, status, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
            cv2.putText(frame, f"Omuz Açı: {shoulder_angle:.1f}/{shoulder_threshold:.1f}", (10, 60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)
            cv2.putText(frame, f"Boyun Açı: {neck_angle:.1f}/{neck_threshold:.1f}", (10, 90), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)
            cv2.putText(frame, f"Kafa Mesafesi: {head_distance:.1f}/{head_distance_threshold:.1f}", (10, 120), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)

    cv2.imshow('Postür Kontrolü', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Kalibrasyon tamamlandi. Omuz: 85.1, Boyun: 32.8, Kafa Mesafesi: 127.8


Versiyon 5 - Veri eğitmeli türü

In [5]:
import cv2
import mediapipe as mp
import numpy as np

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
cap = cv2.VideoCapture(0)

is_calibrated = False
calibration_frames = 0
calibration_shoulder_angles = []
calibration_neck_angles = []
calibration_head_distances = []
CALIBRATION_FRAME_COUNT = 100

def calculate_angle(point1, point2, point3):
    a = np.array(point1)
    b = np.array(point2)
    c = np.array(point3)

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360.0 - angle

    return angle

def calculate_head_distance(ear, shoulder):
    return np.abs(ear[0] - shoulder[0])

def draw_angle(frame, point1, point2, point3, angle, color):
    cv2.putText(frame, str(int(angle)), point2, cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2, cv2.LINE_AA)

def draw_shapes(frame, left_shoulder, right_shoulder, left_ear, right_ear):
    midpoint = ((left_shoulder[0] + right_shoulder[0]) // 2, (left_shoulder[1] + right_shoulder[1]) // 2)

    cv2.line(frame, left_shoulder, midpoint, (255, 255, 255), 2)
    cv2.line(frame, right_shoulder, midpoint, (255, 255, 255), 2)
    
    cv2.line(frame, left_shoulder, left_ear, (255, 255, 255), 2)
    cv2.line(frame, right_shoulder, right_ear, (255, 255, 255), 2)
    
    upper_midpoint = (midpoint[0], midpoint[1] - 100)  
    cv2.line(frame, midpoint, upper_midpoint, (255, 255, 255), 2)
    cv2.line(frame, left_ear, upper_midpoint, (255, 255, 255), 2)
    cv2.line(frame, right_ear, upper_midpoint, (255, 255, 255), 2)

    cv2.circle(frame, left_shoulder, 5, (255, 255, 255), -1)
    cv2.circle(frame, right_shoulder, 5, (255, 255, 255), -1)
    cv2.circle(frame, midpoint, 5, (255, 255, 255), -1)
    cv2.circle(frame, upper_midpoint, 5, (255, 255, 255), -1)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        continue

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        left_shoulder = (int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x * frame.shape[1]),
                         int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y * frame.shape[0]))
        right_shoulder = (int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x * frame.shape[1]),
                          int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y * frame.shape[0]))
        left_ear = (int(landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].x * frame.shape[1]),
                    int(landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].y * frame.shape[0]))
        right_ear = (int(landmarks[mp_pose.PoseLandmark.RIGHT_EAR.value].x * frame.shape[1]), 
                     int(landmarks[mp_pose.PoseLandmark.RIGHT_EAR.value].y * frame.shape[0]))

        shoulder_angle = calculate_angle(left_shoulder, right_shoulder, (right_shoulder[0], 0))
        neck_angle = calculate_angle(left_ear, left_shoulder, (left_shoulder[0], 0))
        head_distance = calculate_head_distance(left_ear, left_shoulder)

        head_tilt_angle = calculate_angle(left_ear, left_shoulder, right_shoulder)
        
        if not is_calibrated and calibration_frames < CALIBRATION_FRAME_COUNT:
            calibration_shoulder_angles.append(shoulder_angle)
            calibration_neck_angles.append(neck_angle)
            calibration_head_distances.append(head_distance)
            calibration_frames += 1
            cv2.putText(frame, f"Kalibrasyon yapılıyor... {calibration_frames}/{CALIBRATION_FRAME_COUNT}", (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
        elif not is_calibrated:
            shoulder_threshold = np.mean(calibration_shoulder_angles) - 3
            neck_threshold = np.mean(calibration_neck_angles) - 3
            head_distance_threshold = np.mean(calibration_head_distances) + 10 
            is_calibrated = True
            print(f"Kalibrasyon tamamlandı. Omuz: {shoulder_threshold:.1f}, Boyun: {neck_threshold:.1f}, Kafa Mesafesi: {head_distance_threshold:.1f}")

        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        draw_shapes(frame, left_shoulder, right_shoulder, left_ear, right_ear)

        midpoint = ((left_shoulder[0] + right_shoulder[0]) // 2, (left_shoulder[1] + right_shoulder[1]) // 2)
        draw_angle(frame, left_shoulder, midpoint, (midpoint[0], 0), shoulder_angle, (255, 0, 0))
        draw_angle(frame, left_ear, left_shoulder, (left_shoulder[0], 0), neck_angle, (0, 255, 0))

        if is_calibrated:
            if shoulder_angle < shoulder_threshold or neck_angle < neck_threshold or head_distance > head_distance_threshold or head_tilt_angle < 30:
                status = "Kötü Postür" 
                color = (0, 0, 255)  
            else:
                status = "İyi Postür"
                color = (0, 255, 0)  

            cv2.putText(frame, turkish_character_fix(status), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
            cv2.putText(frame, f"Omuz Açı: {shoulder_angle:.1f}/{shoulder_threshold:.1f}", (10, 60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)
            cv2.putText(frame, f"Boyun Açı: {neck_angle:.1f}/{neck_threshold:.1f}", (10, 90), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)
            cv2.putText(frame, f"Kafa Mesafesi: {head_distance:.1f}/{head_distance_threshold:.1f}", (10, 120), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)

    cv2.imshow('Postür Kontrolü', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Kalibrasyon tamamlandı. Omuz: 86.6, Boyun: 27.9, Kafa Mesafesi: 108.5


BEST

In [10]:
import cv2
import mediapipe as mp
import numpy as np

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
cap = cv2.VideoCapture(0)

is_calibrated = False
calibration_frames = 0
calibration_shoulder_angles = []
calibration_neck_angles = []
calibration_head_distances = []
calibration_left_shoulder_heights = []
calibration_right_shoulder_heights = []
CALIBRATION_FRAME_COUNT = 100

def calculate_angle(point1, point2, point3):
    a = np.array(point1)
    b = np.array(point2)
    c = np.array(point3)

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360.0 - angle

    return angle

def calculate_distance_y(point1, point2):
    """İki nokta arasındaki dikey mesafeyi hesaplar (y ekseninde)."""
    return np.abs(point1[1] - point2[1])

def calculate_head_distance(ear, shoulder):
    return np.abs(ear[0] - shoulder[0])

def draw_angle(frame, point1, point2, point3, angle, color):
    cv2.putText(frame, str(int(angle)), point2, cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2, cv2.LINE_AA)

def draw_shapes(frame, left_shoulder, right_shoulder, left_ear, right_ear):
    midpoint = ((left_shoulder[0] + right_shoulder[0]) // 2, (left_shoulder[1] + right_shoulder[1]) // 2)

    cv2.line(frame, left_shoulder, midpoint, (255, 255, 255), 2)
    cv2.line(frame, right_shoulder, midpoint, (255, 255, 255), 2)
    
    cv2.line(frame, left_shoulder, left_ear, (255, 255, 255), 2)
    cv2.line(frame, right_shoulder, right_ear, (255, 255, 255), 2)
    
    upper_midpoint = (midpoint[0], midpoint[1] - 100)  
    cv2.line(frame, midpoint, upper_midpoint, (255, 255, 255), 2)
    cv2.line(frame, left_ear, upper_midpoint, (255, 255, 255), 2)
    cv2.line(frame, right_ear, upper_midpoint, (255, 255, 255), 2)

    cv2.circle(frame, left_shoulder, 5, (255, 255, 255), -1)
    cv2.circle(frame, right_shoulder, 5, (255, 255, 255), -1)
    cv2.circle(frame, midpoint, 5, (255, 255, 255), -1)
    cv2.circle(frame, upper_midpoint, 5, (255, 255, 255), -1)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        continue

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        left_shoulder = (int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x * frame.shape[1]),
                         int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y * frame.shape[0]))
        right_shoulder = (int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x * frame.shape[1]),
                          int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y * frame.shape[0]))
        left_ear = (int(landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].x * frame.shape[1]),
                    int(landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].y * frame.shape[0]))
        right_ear = (int(landmarks[mp_pose.PoseLandmark.RIGHT_EAR.value].x * frame.shape[1]), 
                     int(landmarks[mp_pose.PoseLandmark.RIGHT_EAR.value].y * frame.shape[0]))

        shoulder_angle = calculate_angle(left_shoulder, right_shoulder, (right_shoulder[0], 0))
        neck_angle = calculate_angle(left_ear, left_shoulder, (left_shoulder[0], 0))
        head_distance = calculate_head_distance(left_ear, left_shoulder)

        shoulder_imbalance = calculate_distance_y(left_shoulder, right_shoulder)

        head_tilt_angle = calculate_angle(left_ear, left_shoulder, right_shoulder)
        
        if not is_calibrated and calibration_frames < CALIBRATION_FRAME_COUNT:
            calibration_shoulder_angles.append(shoulder_angle)
            calibration_neck_angles.append(neck_angle)
            calibration_head_distances.append(head_distance)
            calibration_left_shoulder_heights.append(left_shoulder[1])
            calibration_right_shoulder_heights.append(right_shoulder[1])
            calibration_frames += 1
            cv2.putText(frame, f"Kalibrasyon yapılıyor... {calibration_frames}/{CALIBRATION_FRAME_COUNT}", (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
        elif not is_calibrated:
            shoulder_threshold = np.mean(calibration_shoulder_angles) - 3
            neck_threshold = np.mean(calibration_neck_angles) - 3
            head_distance_threshold = np.mean(calibration_head_distances) + 10 
            shoulder_imbalance_threshold = np.abs(np.mean(calibration_left_shoulder_heights) - np.mean(calibration_right_shoulder_heights)) + 5  # Omuzlar arasındaki yükseklik farkı için eşik
            is_calibrated = True
            print(f"Kalibrasyon tamamlandı. Omuz: {shoulder_threshold:.1f}, Boyun: {neck_threshold:.1f}, Kafa Mesafesi: {head_distance_threshold:.1f}, Omuz Farkı: {shoulder_imbalance_threshold:.1f}")

        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        draw_shapes(frame, left_shoulder, right_shoulder, left_ear, right_ear)

        midpoint = ((left_shoulder[0] + right_shoulder[0]) // 2, (left_shoulder[1] + right_shoulder[1]) // 2)
        draw_angle(frame, left_shoulder, midpoint, (midpoint[0], 0), shoulder_angle, (255, 0, 0))
        draw_angle(frame, left_ear, left_shoulder, (left_shoulder[0], 0), neck_angle, (0, 255, 0))

        if is_calibrated:
            if shoulder_angle < shoulder_threshold or neck_angle < neck_threshold or head_distance > head_distance_threshold or head_tilt_angle < 30 or shoulder_imbalance > shoulder_imbalance_threshold:
                status = "Kötü Postür" 
                color = (0, 0, 255)  
            else:
                status = "İyi Postür"
                color = (0, 255, 0)  

            cv2.putText(frame, status, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
            cv2.putText(frame, f"Omuz Açı: {shoulder_angle:.1f}/{shoulder_threshold:.1f}", (10, 60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)
            cv2.putText(frame, f"Boyun Açı: {neck_angle:.1f}/{neck_threshold:.1f}", (10, 90), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)
            cv2.putText(frame, f"Kafa Mesafesi: {head_distance:.1f}/{head_distance_threshold:.1f}", (10, 120), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)
            cv2.putText(frame, f"Omuz Dengesizliği: {shoulder_imbalance:.1f}/{shoulder_imbalance_threshold:.1f}", (10, 150), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)

    cv2.imshow('Postür Kontrolü', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Kalibrasyon tamamlandı. Omuz: 87.6, Boyun: 25.5, Kafa Mesafesi: 99.5, Omuz Farkı: 7.9
